### Ok, the first requirement is 

1. La universidad opera de forma presencial y con planes de estudios semestrales. Se debe registrar la información de cada plan de estudios como: nombre, clave, duración en semestres, número de créditos totales, estatus (vigente o no).

This seems to just be a table. The format is

CREATE TABLE study_plan(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    code VARCHAR(50),
    duration INTEGER,
    credits INTEGER,
    status BOOL,
);

I don't think code and ID are the same, as the code probably has implicit information about what subject is related to, the undergraduate that is imparted in, etc. As it probably has both letters and numbers I think it is better to just leave it as VARCHAR. 

I don't know if you declare a boolean variable in postgreSQL with BOOL, but I will know once I run the code. 

As this table doesn't has any foreign key is a strong entity; one of the first ones I will create in the database. 

### The second requirement is 

2. Se debe almacenar la información de todas las materias comprendidas en cada plan de estudios: clave, nombre, número de horas, número de horas teóricas, numéro de horas de práctica, número de créditos, área de especialización.

CREATE TABLE subjects(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    hours INTEGER,
    theorical_hours INTEGER,
    practical_hours INTEGER,
    credits INTEGER,
    area 
);

In area I will most probably need a foreign key. It seems that the relationship is one to one. I think that I should put the rule here that no attribute of this table can be null, but also that the arithmetic sum of the 'theorical_hours' and 'practical_hours' must be equal or less than 'hours'. 

### The third requirement is 

3. Se debe almacenar la información personal del alumno como: nombre completo, nacionalidad, curp, fecha de nacimiento, correo electrónico personal, teléfono, dirección, estado civil, tipo de sangre, numero de seguridad social. No es posible que dos estudiantes compartan correo electrónico.

Another table

CREATE TABLE students(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    nacionality VARCHAR(10),
    curp VARCHAR(30), 
    birth_date DATE,
    email TEXT UNIQUE,
    telephone VARCHAR(15),
    city VARCHAR(100), 
    state VARCHAR(100), 
    street VARCHAR(100),
    external_number INTEGER,
    zip_code INTEGER,
);

Creating tables for the last names seems excessive, but useful at the same time... not really, I don't think having separate tables for the last names will help more than just doing a normal SQL search. My doubt is how can I ensure that the students will fill out the information in the 'Name PaternalName MaternalName' standard? 

In nacionality I have doubt too. I should probably have a list of codes that map all the nacionalities in the world, and nationality as an attribute to just be a number. 

The same goes for the CURP... I could use regex to ensure at least the curp that the student wrote makes sense, as I have already done it for the subject of formal languages. 

I will stop worrying for that now, as it seems that I can get into the same trouble with all the students' attributes. 

I know I need to pay special attention to the adress in order to comply with 3rd Normal Form. It seems that the way to do so is by breaking adress into composite values such as street, external number and zip code. 

It seems that for email I need to use TEXT in addition to a CHECK constraint. Indeed I can use regex for the email or other camps. 

As I have already wrote quite a bit of text I will start with the code. First importing psycopg2 for postgreSQL. 















In [24]:
import psycopg
from dotenv import load_dotenv
import os

load_dotenv()

def conn_to_postgres(instructions):
    conn = None

    try:
        conn = psycopg.connect(
            host="localhost",
            dbname="unam_database",
            user="schronding",
            password=os.getenv('PASSWORD'),
            port="5432", 
        ) 

        conn.autocommit = True
        cur = conn.cursor()

        cur.execute(instructions)


        cur.close()
        return conn
    
    except(Exception, psycopg.DatabaseError) as e:
        print(f"Error: {e}")
        if conn:
            cur.close()
        return None



Before executing the function (which I think it is wrong as I probably confuse what the difference between the connection and cursor is), I need to create the database... but I don't remember the syntax 

**postgres=# CREATE DABATASE unam_database;**

*ERROR:  syntax error at or near "DABATASE"*

*LINE 1: CREATE DABATASE unam_database;*

I wrote DATABASE wrong... Now it works. 

It seems that while the syntax is correct I am not able to set schronding as the owner of 'unam_database'. I think it probably has to do with me not creating this role for this specific database. 

postgres=# CREATE DABATASE unam_database;

ERROR:  syntax error at or near "DABATASE"

LINE 1: CREATE DABATASE unam_database;

To check the roles I can use \du and \du+. 

I am having trouble now even entering to the postgres terminal 

**sudo psql -U postgres**

psql: error: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: FATAL:  Peer authentication failed for user "postgres"

It seems that I can reconfigure the 'pg_hba.conf' file to trust my local computer... but I don't know where is located. It doesn't work with 'whereis' on the terminal. 

I will just use sudo -u postgres psql. 

It seems I am still denied permision but the terminal appeared again

**sudo -u postgres psql**

could not change directory to "/home/schronding/repos/UNAM_database": 

Permission denied

psql (15.14 (Debian 15.14-0+deb12u1))

It wasn't that my role didn't exist, as it already does 

**postgres=# CREATE ROLE schronding;**

ERROR:  role "schronding" already exists

With the command 

**ALTER ROLE schronding WITH LOGIN PASSWORD MarioYChuyBFF**

the postgres terminal changed from 'postgres=#' to 'postgres-#'. I don't know what it means the change from = to -. It seems that the difference is that when it goes to - it means that you haven't finished executing the previous command. I think this could be useful to create tables with multiple attributes. 

I think I should probably use the library of running terminal scripts in python, as I think that would cut the need to copy all my commands and their output. The library's name is 'subprocess'. It might not work, as there are two levels of 'depth' with the postgres terminal. 

In [47]:
import subprocess

study_plan_table = """
DROP TABLE IF EXISTS study_plans CASCADE; 

CREATE TABLE study_plans(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    code VARCHAR(50),
    duration INTEGER,
    credits INTEGER,
    status BOOLEAN NOT NULL
);"""

conn_to_postgres(study_plan_table)

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f41399a0>

I had some errors

Error: invalid connection option "name"

Multiple connection attempts failed. All failures were:
- host: 'localhost', port: '5432', hostaddr: '::1': invalid connection option "name"

- host: 'localhost', port: '5432', hostaddr: '127.0.0.1': invalid connection option "name"

The thing with psycopg is that it seems that the order of the attributes matter, even though I am explicitly telling the parameters by name. While psycopg seems to be a better version than psycopg2 I might be getting all of these problems just by the difference in versions, as the conn that I remembered was with psycopg2. 

Also prior to that I forgot that in the except clause we need 'Exception', not 'Error'. Also that the port must be a string, not a number. 

It is interesting, because when I put name it throwed an error, when I changed the attribute to database it did too. It seems that the attribute itself is the problem. 

I don't know what the problem is... 

It was just that the correct name attribute was 'dbname'... 

At least now I get a different error, which I understand

Error: connection failed: connection to server at "127.0.0.1", port 5432 failed: fe_sendauth: no password supplied
Multiple connection attempts failed. All failures were:
- host: 'localhost', port: '5432', hostaddr: '::1': connection failed: connection to server at "::1", port 5432 failed: fe_sendauth: no password supplied
- host: 'localhost', port: '5432', hostaddr: '127.0.0.1': connection failed: connection to server at "127.0.0.1", port 5432 failed: fe_sendauth: no password supplied

What I don't understand is why the password isn't correctly retrieved. Could it be that by being in a Jupyter notebook, the getenv method doesn't recognize that there is a hidden file .env in the local environment? 

Indeed when I try to get the password it doesn't retrieve it

In [ ]:
#print(os.getenv("PASSWORD"))

MarioYChuyBFF


I now see what the problem was... I didn't include the 'load_dotenv()' function. 

Now it finally works 

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f52178c0>

In [28]:
print(conn_to_postgres("SELECT ALL"))

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f4bdf610>


What I wanted to see was if there was a good formated answer for when I made a SQL request. I will research it later, I think I can continue with the second table. Thinking it more slowly I do think that while on the ENES there might be different versions of the same subject, for example 'Probabilidad y Estadistica' for technology and other for aeroespacial engineering, I think it makes sense that one subject can belong to many study plans, creating a connection one to many. 

Part of me thinks that many to many is correct, but is redundant. Can one study plan be involved in a lot of subjects? Yes, but is it relevant? Do I need to write the intermediate table? For now I will do it. 

As the intermediate or pivot table has foreign keys is weak, but subjects is weak too. Is there any difference by which I create first? 

In [48]:
plan_subjects_pivot_table = """
DROP TABLE IF EXISTS plan_subjects CASCADE;

CREATE TABLE plan_subjects(
    plan_id FOREIGN KEY id REFERENCES plan(id),
    subject_id FOREIGN KEY id REFERENCES subject(id)
);
"""

conn_to_postgres(plan_subjects_pivot_table)

Error: syntax error at or near "FOREIGN"
LINE 5:     plan_id FOREIGN KEY id REFERENCES plan(id),
                    ^


As I got the error

Error: syntax error at or near "FOREIGN"
LINE 5:     plan_id FOREIGN KEY id REFERENCES plan(id),
                    ^

I will try to first create the subjects table without the foreign key, and then execute the pivot table. 

In [49]:
subjects_table = """
DROP TABLE IF EXISTS subjects CASCADE;

CREATE TABLE subjects(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    theorical_hours INTEGER,
    practical_hours INTEGER,
    hours INTEGER,
    credits INTEGER,
    area VARCHAR(100),
    FOREIGN KEY (area) REFERENCES study_plans(name),
    CONSTRAINT hours CHECK (theorical_hours + practical_hours <= hours)
);"""

It seems I got the error 

Error: syntax error at or near ")"
LINE 11: );
         ^

Because I left the last attribute with a comma at the end. When I removed it I got a sucessful message again. 

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f52db9b0>

The error remains in the pivot table though. I will continue with the students table while I think how to solve that error

In [50]:
students_table ="""
DROP TABLE IF EXISTS students CASCADE;

CREATE TABLE students(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    nacionality VARCHAR(10),
    curp VARCHAR(30), 
    birth_date DATE,
    email TEXT UNIQUE,
    telephone VARCHAR(15),
    city VARCHAR(100), 
    state VARCHAR(100), 
    street VARCHAR(100),
    external_number INTEGER,
    zip_code INTEGER,
    civil_state BOOLEAN,
    blood_type VARCHAR(10),
    NSS INTEGER
);"""

conn_to_postgres(students_table)

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f4139b80>

It seems it worked! 

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f41456a0>

As I don't like that nacionality is VARCHAR I will use the lookup table of formal languages so I can change the type to a number. 

### Nacionalities

        paises.add("MX : MÉXICO");
        paises.add("DE : ALEMANIA");
        paises.add("AD : ANDORRA");
        paises.add("AR : ARGENTINA");
        paises.add("AU : AUSTRALIA");
        paises.add("AT : AUSTRIA");
        paises.add("BE : BÉLGICA");
        paises.add("BO : BOLIVIA");
        paises.add("BR : BRASIL");
        paises.add("CA : CANADÁ");
        paises.add("CL : CHILE");
        paises.add("CN : CHINA");
        paises.add("CO : COLOMBIA");
        paises.add("KR : COREA DEL SUR");
        paises.add("CR : COSTA RICA");
        paises.add("CU : CUBA");
        paises.add("DK : DINAMARCA");
        paises.add("EC : ECUADOR");
        paises.add("EG : EGIPTO");
        paises.add("SV : SALVADOR, EL");
        paises.add("ES : ESPAÑA");
        paises.add("US : ESTADOS UNIDOS DE AMÉRICA");
        paises.add("PH : FILIPINAS");
        paises.add("FI : FINLANDIA");
        paises.add("FR : FRANCIA");
        paises.add("GR : GRECIA");
        paises.add("GT : GUATEMALA");
        paises.add("HN : HONDURAS");
        paises.add("IN : INDIA");
        paises.add("ID : INDONESIA");
        paises.add("IE : IRLANDA");
        paises.add("IL : ISRAEL");
        paises.add("IT : ITALIA");
        paises.add("JM : JAMAICA");
        paises.add("JP : JAPON");
        paises.add("MA : MARRUECOS");
        paises.add("NI : NICARAGUA");
        paises.add("NO : NORUEGA");
        paises.add("NZ : NUEVA ZELANDA");
        paises.add("PA : PANAMA");
        paises.add("PY : PARAGUAY");
        paises.add("PE : PERÚ");
        paises.add("PL : POLONIA");
        paises.add("PT : PORTUGAL");
        paises.add("PR : PUERTO RICO");
        paises.add("GB : REINO UNIDO");
        paises.add("CZ : CHECA, REPÚBLICA");
        paises.add("DO : DOMICANA, REPÚBLICA");
        paises.add("RU : RUSIA");
        paises.add("ZA : SUDÁFRICA");
        paises.add("SE : SUECIA");
        paises.add("CH : SUIZA");
        paises.add("TH : TAILANDIA");
        paises.add("TR : TURQUÍA");
        paises.add("UA : UCRANIA");
        paises.add("UY : URUGUAY");
        paises.add("VE : VENEZUELA");

### Regex 

While it is in Java I will store here the regex in case they come in handy later down the line 

static {
        REGEX_MAP.put("Nombre(s)", "^[A-ZÁÉÍÓÚÑ ]{1,20}$");
        REGEX_MAP.put("Apellido Paterno", "^[A-ZÁÉÍÓÚÑ ]{1,15}$");
        REGEX_MAP.put("Apellido Materno", "^[A-ZÁÉÍÓÚÑ ]{1,15}$");
        REGEX_MAP.put("Fecha de nacimiento", "^(?:0?[1-9]|[12]\\d|3[01])([-/])(?:0?[1-9]|1[0-2])\\1(?:(?:\\d{2})?\\d{2})$");
        REGEX_MAP.put("RFC", "^[A-Z]{4}\\d{6}[A-Z0-9]{3}$");
        REGEX_MAP.put("CURP", "^[A-Z]{4}\\d{6}[HM][A-Z]{2}[A-Z]{3}[A-Z0-9]{2}$");
        REGEX_MAP.put("Calle", "^[A-Z0-9 ]+$");
        REGEX_MAP.put("Número Ext.", "^\\d+$");
        REGEX_MAP.put("Colonia", "^[A-Z0-9 ]+$");
        REGEX_MAP.put("Código Postal", "^\\d{5}$");
        REGEX_MAP.put("Teléfono1", "^\\d{3}$");
        REGEX_MAP.put("Teléfono2", "^\\d{3}$");
        REGEX_MAP.put("Teléfono3", "^\\d{4}$");
        REGEX_MAP.put("Correo electrónico", "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9]+(?:[.-][a-zA-Z0-9]+)*\\.[a-zA-Z]{2,}$");
        REGEX_MAP.put("Contraseña", "^(?=.*[a-z])(?=.*[A-Z])(?=(?:.*\\d){2})(?=.*[\\W_]).{8,}$");
        REGEX_MAP.put("Tarjeta1", "^\\d{4}$");
        REGEX_MAP.put("Tarjeta2", "^\\d{4}$");
        REGEX_MAP.put("Tarjeta3", "^\\d{4}$");
        REGEX_MAP.put("Tarjeta4", "^\\d{4}$");
    }

### Continue to solve the pivot table

It seems that the problem was that I wrote FOREIGN KEY before specifying the type. It is actually backwards: first I declare the type, then I tell postgres that it is a foreign key. 


In [51]:
plan_subjects_pivot_table = """
DROP TABLE IF EXISTS plan_subjects CASCADE;

CREATE TABLE plan_subjects(
    plan_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (plan_id) REFERENCES study_plans(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
);
"""

conn_to_postgres(plan_subjects_pivot_table)

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f413a5d0>

I forgot to put the area of specialization in the subjects table, and as I wanted to update it with the name of the study plan it belongs to (I suppose that is what is meant, maybe it means that it is from the 4 areas of the unam: mathematics, biology, social and humanities). I think there shouldn't be a problem to drop on cascade, as I still do not have data; and when I have it I should store it in an outside csv to not struggle with it later. 

Error: cannot drop table subjects because other objects depend on it
DETAIL:  constraint plan_subjects_subject_id_fkey on table plan_subjects depends on table subjects
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

It seems that I just need to put CASCADE at the end of the command of dropping the table. 

It worked! But while I was dropping the subjects table I got the error 

Error: there is no unique constraint matching given keys for referenced table "study_plans"

And this makes sense to me: how can the database know from all the available names the ones I want to reference? I supposed that when I inserted the data the name itself would look for any direct match with the study_plans table... 



### The 4th requirement is

4. Se debe registrar la información académica previa del estudiante como: institución de origen, tipo de bachillerato, promedio del último bachillerato, fecha de egreso, ubicación.

I assume this will be a table one to many (one student to all the attributes in the information). 


In [ ]:

student_info_table = """
DROP TABLE IF EXISTS student_info CASCADE;

CREATE TABLE student_info(
    origin VARCHAR(100),
    type_school TEXT,
    average FLOAT,
    graduation DATE,
    location TEXT
);
"""